<a href="https://colab.research.google.com/github/jazminenecessary-dot/sds510/blob/main/SDS510_Module-5-Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import os
import re
import pandas as pd
from string import punctuation
from datetime import datetime
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

english_stopwords = stopwords.words('english') + list(punctuation)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
game = pd.read_json('jeopardy.json')

In [11]:
questions = make_wordlist(game['question'])
answers = make_wordlist(game['answer'])

In [12]:
def make_wordlist(text_series):

    word_lists = []
    lemmatizer = WordNetLemmatizer()
    for text in text_series:
        # Tokenize, convert to lowercase, remove punctuation/numbers, remove stopwords, and lemmatize
        tokens = [
            lemmatizer.lemmatize(word.lower())
            for word in word_tokenize(text)
            if word.isalpha() and word.lower() not in english_stopwords
        ]
        word_lists.append(tokens)
    return word_lists

In [13]:
game

,category,air_date,question,value,answer,round,show_number
0,HISTORY,2004-12-31,"'For the last 8 years of his life, Galileo was...",$200,Copernicus,Jeopardy!,4680
1,ESPN's TOP 10 ALL-TIME ATHLETES,2004-12-31,'No. 2: 1912 Olympian; football star at Carlis...,$200,Jim Thorpe,Jeopardy!,4680
2,EVERYBODY TALKS ABOUT IT...,2004-12-31,'The city of Yuma in this state has a record a...,$200,Arizona,Jeopardy!,4680
3,THE COMPANY LINE,2004-12-31,"'In 1963, live on ""The Art Linkletter Show"", t...",$200,McDonald\'s,Jeopardy!,4680
4,EPITAPHS & TRIBUTES,2004-12-31,"'Signer of the Dec. of Indep., framer of the C...",$200,John Adams,Jeopardy!,4680
...,...,...,...,...,...,...,...
216925,RIDDLE ME THIS,2006-05-11,'This Puccini opera turns on the solution to 3...,$2000,Turandot,Double Jeopardy!,4999
216926,"""T"" BIRDS",2006-05-11,'In North America this term is properly applie...,$2000,a titmouse,Double Jeopardy!,4999
216927,AUTHORS IN THEIR YOUTH,2006-05-11,"'In Penny Lane, where this ""Hellraiser"" grew u...",$2000,Clive Barker,Double Jeopardy!,4999
216928,QUOTATIONS,2006-05-11,"'From Ft. Sill, Okla. he made the plea, Arizon...",$2000,Geronimo,Double Jeopardy!,4999


In [16]:
hardquestions = [1 if (x is not None and int(str(x).replace('$', '').replace(',', '')) >= 1000) else 0 for x in game['value']]

I decided to classify hard questions by the value of the question. For hard questions, they need to be worth at least $1000

In [22]:
game['is_hard_question'] = hardquestions
display(game[['question', 'answer', 'is_hard_question']])

,question,answer,is_hard_question
0,"'For the last 8 years of his life, Galileo was...",Copernicus,0
1,'No. 2: 1912 Olympian; football star at Carlis...,Jim Thorpe,0
2,'The city of Yuma in this state has a record a...,Arizona,0
3,"'In 1963, live on ""The Art Linkletter Show"", t...",McDonald\'s,0
4,"'Signer of the Dec. of Indep., framer of the C...",John Adams,0
...,...,...,...
216925,'This Puccini opera turns on the solution to 3...,Turandot,1
216926,'In North America this term is properly applie...,a titmouse,1
216927,"'In Penny Lane, where this ""Hellraiser"" grew u...",Clive Barker,1
216928,"'From Ft. Sill, Okla. he made the plea, Arizon...",Geronimo,1


In [24]:
print(datetime.now().strftime("%H:%M:%S"))
allquestions = [' '.join(word_list) for word_list in questions]
print(datetime.now().strftime("%H:%M:%S"))

21:36:51
21:36:52


In [29]:
df = pd.DataFrame({'question': allquestions, # Use allquestions (joined strings) instead of questions (lists of words)
                   'answer': answers,
                   'is_hard_question': hardquestions})

In [30]:
X_train, X_test, y_train, y_test = train_test_split(df.question, df.is_hard_question, # Swap arguments: df.question is X, df.is_hard_question is y
                                                    random_state=1)

In [32]:
tfidf_vectorizer = TfidfVectorizer(use_idf=True)
X_train_tf= tfidf_vectorizer.fit_transform(X_train)
X_test_tf = tfidf_vectorizer.transform(X_test)

In [33]:
naive_bayes = MultinomialNB()
naive_bayes.fit(X_train_tf, y_train)
predictions = naive_bayes.predict(X_test_tf)

In [36]:
print('Accuracy: ', accuracy_score(y_test, predictions))

Accuracy:  0.7152840521453727
